实验四：区间树上重叠区间的查找算法
实验要求：
	对实验三的红黑树算法进行修改，使其成为一颗区间数，并实现区间树上的重叠区间查找算法。
	程序输入：
		1、生成区间树
			文件名： insert.txt
			文件格式：第一行为待插入数据的个数，第二行之后每一行表示一个区间
		注：1）初始时树应为空。
			2）按顺序插入， 例如，对于下图的数据，插入顺序应为 [50, 60]，[20, 25]， [70, 90]
 
		2、待查询区间应由控制台输入
	程序输出： 
		控制台直接打印查找结果(如果存在多个区间与query重叠, 需输出所有与query重叠的区间)。

实验报告要求：
	1）实验内容
	2）实验目的
	3）区间树的数据结构
	4）源码 + 注释
	5）算法测试结果
	6）实验过程中遇到的困难及收获




提交方式：
	提交内容：
		1、源码
		2、实验报告
		2021年12月15日  23：59：59 前


In [31]:
#定义区间树
class Interval:
    def __init__(self, low, high):
        self.low = low
        self.high = high
    def __str__(self) -> str:
        return '[' + str(self.low) + ',' + str(self.high) + ']'

class IntervalTree(object):
    def __init__(self):
        self.nil = IntervalTreeNode(Interval(0,0))
        self.root = self.nil

class IntervalTreeNode(object):
    def __init__(self,interval):
        self.key = interval.low
        self.left = None
        self.right = None
        self.p = None
        self.color = 'black'
        self.size = None
        self.max = interval.high
        self.int = interval
        self.case = 0 # 默认case为0，取值为0-6



In [32]:
#插入
def ITInsert(T, z):
    y = T.nil
    x = T.root
    while x != T.nil:
        y = x
        if z.key < x.key:
            x = x.left
        else:
            x = x.right
    z.p = y
    if y == T.nil:
        T.root = z
    elif z.key < y.key:
        y.left = z
    else:
        y.right = z
    z.left = T.nil
    z.right = T.nil
    z.color = 'red'
    ITInsertFixup(T, z)
    return z.key, '颜色为', z.color

def ITInsertFixup(T, z):
    while z.p.color == 'red':
        if z.p == z.p.p.left:  #若“z的父节点”是“z的祖父节点的左孩子”
            y = z.p.p.right
            if y.color == 'red':        #case 1 叔为红色
                z.p.color = 'black'
                y.color = 'black'
                z.p.p.color = 'red'
                z = z.p.p
                z.case = 1
            else:
                if z == z.p.right: 
                    z.case = 2     # Case 2条件：叔叔是黑色，且当前节点是右孩子
                    z = z.p             
                    LeftRotate(T, z)  
                else:   
                    z.case = 3 
                z.p.color = 'black'     #Case 3条件：叔叔是黑色，且当前节点是左孩子。(01) 将“父节点”设为“黑色”。
                z.p.p.color = 'red'     
                RightRotate(T,z.p.p)                    
        else:
            y = z.p.p.left
            if y.color == 'red':
                z.p.color = 'black'
                y.color = 'black'
                z.p.p.color = 'red'
                z = z.p.p
                z.case = 4
            else:
                if z == z.p.left:
                    z = z.p
                    RightRotate(T, z)
                    z.case = 5
                else:
                    z.case = 6
                z.p.color = 'black'
                z.p.p.color = 'red'
                LeftRotate(T, z.p.p)       
    T.root.color = 'black'


In [33]:
#左旋转
def LeftRotate(T, x):
    y = x.right
    x.right = y.left
    if y.left != T.nil:
        y.left.p = x
    y.p = x.p
    if x.p == T.nil:
        T.root = y
    elif x == x.p.left:
        x.p.left = y
    else:
        x.p.right = y
    y.left = x
    x.p = y
    y.max = x.max
    x.max = max(x.int.high, x.left.max, x.right.max)

#右旋转
def RightRotate(T, x):
    y = x.left
    x.left = y.right
    if y.right != T.nil:
        y.right.p = x
    y.p = x.p
    if x.p == T.nil:
        T.root = y
    elif x == x.p.right:
        x.p.right = y
    else:
        x.p.left = y
    y.right = x
    x.p = y
    y.max = x.max
    x.max = max(x.int.high, x.left.max, x.right.max)

In [34]:
def IS_OVERLAP(x,y):
    if x.low>=y.high:
        return False
    elif x.high<=y.low:
        return False
    return True

In [35]:
import queue
def BFS(T,res,i):
    q = queue.Queue()
    q.put(T.root)
    while not q.empty():
        x = q.get()
        if IS_OVERLAP(x.int,i):
            res.append(x)
        if x.left!=None:
            q.put(x.left)
        
        if x.right!=None:
            q.put(x.right)

In [36]:

def INTERVAL_SEARCH(T, i):
    x = T.root
    while x != T.nil and not IS_OVERLAP(i,x.int):
        # print(x.left)
        if x.left != T.nil and x.left.max > i.low:
            x = x.left
        else:
            x = x.right
    # res.append(x)
    # if x.int.high<i.high:
    #     INTERVAL_SEARCH(T,Interval(x.int.high,i.high),res)
    # elif x.int.low>i.low:
    #     res.append(INTERVAL_SEARCH(T,Interval(i.low,x.int.low),res))
    # else:
    return x

In [37]:
#读取文件
def load_data():
    file=open("D:\Mis\project\python\dataMine\data\insert2.txt")    
    p = file.readlines()
    l=[]
    for i in range(1,len(p)):
        p[i] = p[i].split(" ")
        l.append(Interval(int (p[i][0]),int (p[i][1])))
    file.close()
    return l

In [38]:
def INTERVAL_SEARCH_D(T,i,res):
    x = T.root
    while x != T.nil and not IS_OVERLAP(i,x.int):#若不是叶子节点且不重叠，继续搜索子树
        if x.left != T.nil and x.left.max > i.low:
            x = x.left #当x左节点max>区间low是搜索x左子树
        else:
            x = x.right #否则搜索右子树
    if x.int.low<= i.low and x.int.high>=i.high:
        return #递归终止条件，如果x与i无剩余不重叠的区间时，递归结束
    if x==T.nil:
        return # 递归终止条件，x为叶子节点时终止
    res.append(x)
    if x.int.high<=i.high: #递归查询x与i的剩余不重叠区间，当不重叠区间位于左部分
        p =Interval(x.int.high,i.high)
        INTERVAL_SEARCH_D(T,p,res)
        
    if x.int.low>=i.low:#递归查询x与i的剩余不重叠区间，当不重叠区间位于右部分
        p =Interval(i.low,x.int.low)
        INTERVAL_SEARCH_D(T,p,res)


In [39]:
def INTERVAL_SEARCH_DE(T,x,i,res):
    while(x != None or x.int):
        if x.high<=i.high:
            res.append(INTERVAL_SEARCH(T,Interval(x.high,i.high)))
        elif x.low>=i.low:
            res.append(INTERVAL_SEARCH(T,Interval(i.low,x.low)))


In [40]:
nodes = load_data()
T = IntervalTree()
res = []
for node in nodes:
    print(node)
    ITInsert(T,IntervalTreeNode(node))
res = []
i = Interval(55,100)
print("查询区间",i)
BFS(T,res,i)
print("查询结果BFS")
for item in res:
    print(item.int)
x = INTERVAL_SEARCH(T,i)
r.append(x)
r = []
INTERVAL_SEARCH_D(T,i,r)
print("查询结果递归")
for item in r:
    print(item.int)

[50,60]
[20,25]
[70,90]
[15,22]
[30,32]
[25,28]
[35,40]
[32,34]
查询区间 [55,100]
查询结果BFS
[50,60]
[70,90]
查询结果递归
[50,60]
[70,90]
